In [12]:
#import pandas as pd
#import string
#from matplotlib import rc

fontsize = 12

# Make default font serif
rc('font', family=b'serif', size=fontsize)
# Hide legend frame, change size, make only one scatter marker show up
rc('legend', borderpad=0.,labelspacing=0.25, fontsize=(fontsize - 1.), frameon=False, numpoints=1)
# Default line width (e.g. for plot()) should be thinner than 1 point
rc('patch', linewidth=0.75)
rc('lines', linewidth=0.75)

# BW1484 Core I Output

In [13]:
k_ref = 1.00020

### Load and parse the file

In [17]:
import re
import os
import numpy as np

# name of the file containing the output we want to process; 
# figure out how to make this a command line option later
out_file='bw1484_small.out' #$1
full_out_file = os.path.join('/home/slayer/Dropbox/Software/Exnihilo/Calculations/bw1484/',
                            out_file)
if os.path.isfile(full_out_file):
    f = open(full_out_file, 'r')
else:
    raise IOError('could not find file', full_out_file)

## Pairs of output variables that are in the header / problem specification
keys = ['problem_name', 'num cells I', 'num cells J', 'num cells K', 'cells', 'mesh ',
        'num_blocks_i', 'num_blocks_j', 'num_z_blocks', 'blocks', 'num_groups', 
        'num upscatter groups', 'num_sets', 'domains', 'Pn_order', 'azimuthals_octant', 
        'polars_octant', 'eq_set', 'DoF', 'tolerance', 'L2_tolerance', 'k_tolerance',
        'up_tol', 'eigen_solver', 'mg_preconditioner', 'relax_count', 'num_v_cycles', 
        'relax_weight', 'ml_Pn', 'ml_az', 'ml_oct']
vals = {key: None for key in keys}

# set flags and keys to use for items with duplicate entries
ml_flag = False
ml_keys = ['Pn_order', 'azimuthals_octant', 'polars_octant']
ml_doubles = {key: None for key in ml_keys}
ml_cnt = 0

up_db = False
doubles = {'tolerance' : None}

for line in f:
    
    # check for the duplicate items before processing the next stuff
    if ml_flag == True: 
        if ml_cnt != 3:
            for ml_key in ml_doubles:
                if re.search(ml_key, line):
                    if ml_doubles[ml_key] == None:
                        data = line.split(ml_key)
                        ml_doubles[ml_key] = data[1].strip() 
                        ml_cnt = ml_cnt + 1
        else:
            ml_flag = False
    if up_db == True:
        if re.search('tolerance', line):
            if doubles['tolerance'] == None:
                data = line.split('tolerance')
                doubles['tolerance'] = data[1].strip() 
                up_db = False
        
    for key in vals: 
        # check for things to set the flags
        if re.search('multilevel_quad_db', line):
            ml_flag = True
        if re.search('upscatter_db', line):
            up_db = True   
        # now look for the rest of the keys
        if re.search(key, line):
            data = line.split(key)
            vals[key] = data[1].strip()  
        
# calculate things we want in the spreadsheet
vals['cells'] = int(vals['num cells I']) * int(vals['num cells J']) * int(vals['num cells K'])
vals['blocks'] = int(vals['num_blocks_i']) * int(vals['num_blocks_j'])
vals['domains'] = int(vals['blocks']) * int(vals['num_sets'])
# set doubles
vals['ml_Pn'] = ml_doubles['Pn_order']
vals['ml_az'] = ml_doubles['azimuthals_octant']
vals['ml_oct'] = ml_doubles['polars_octant']
vals['up_tol'] = doubles['tolerance']

#------------------------------------------------------
# Solution information: iteration, time, etc.

# put file back to beginning
f.seek(0,0)

# one time keys
run_keys = ['procs-per-node-x', 'procs-per-node-y', ]

# Iteration information
key_map = {'fxd_iters' : 'Iterations =', 'fxd_res' : '[ 0 ] =', 
           'eigen_iters' : 'Eigenvalue Iteration', 'rel_err_k' : 'relative error', 
           'abs_err_k' : 'absolute error'}
iter_dict = {'fixed_source' : {'Iterations =' : []},#, '[ 0 ] =' : []},
             'eigenvalue' : {'Eigenvalue Iteration    ' : [], 'relative error in k-eff of  ' : [],
                             'absolute error in k-eff of  ' : []}}
# Loop through the file for iteration information
for line in f:
    
    # Fixed source iterations
    for key in iter_dict['fixed_source']:
        if re.search(key, line):
            data = line.split(key)
            num = data[1].split(' ')
            iter_dict['fixed_source'][key].append(num[1].strip())
    # Eigenvalue iterations
    for key in iter_dict['eigenvalue']:
        if re.search(key, line):
            data = line.split(key)
            num = data[1].split(' ')
            iter_dict['eigenvalue'][key].append(num[0].strip())        
print iter_dict
    

{'eigenvalue': {'absolute error in k-eff of  ': ['6.062333e-02', '9.511580e-01', '2.231729e-02', '1.382703e-02', '1.704740e-02', '3.930580e-02', '1.805493e-02', '3.761314e-02', '9.377759e-01', '9.591924e-01', '1.726519e-02', '1.675996e-02', '1.680559e-02', '3.872116e-02', '9.376491e-01', '9.598944e-01', '1.756059e-02', '1.693256e-02', '1.665289e-02', '3.892106e-02', '9.382343e-01', '9.603662e-01', '1.751200e-02', '1.703752e-02', '1.660935e-02', '3.899721e-02', '9.384463e-01', '9.605389e-01', '1.749348e-02', '1.707629e-02', '1.659436e-02', '3.902436e-02', '9.385240e-01', '9.606024e-01', '1.748677e-02', '1.709065e-02', '1.658913e-02', '3.903402e-02', '9.385530e-01', '9.606261e-01', '1.748430e-02', '1.709610e-02', '1.658734e-02', '3.903745e-02', '9.385641e-01', '9.606353e-01', '1.748338e-02', '1.709824e-02', '1.658674e-02', '3.903867e-02', '9.385685e-01', '9.606390e-01', '1.748302e-02', '1.709912e-02', '1.658656e-02', '3.903911e-02', '9.385704e-01', '9.606405e-01', '1.748288e-02'], 'relat

### print information to csv

In [3]:
import csv
with open('output.csv', 'wb') as csvfile:
    # write the header / problem specification information
    writer = csv.DictWriter(csvfile, fieldnames = keys, dialect='excel', 
                            restval='',extrasaction='ignore')
    #if not csv.Sniffer().has_header(csvfile.read(1024)):
    writer.writeheader()
    writer.writerow(vals)
    
    # Now write the iteration and timing information
    